# Pytorch Datasets

Goal: Setting up pytorch with the datasets so it will train to identify a dog or cat

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

# for visualization
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

c:\Users\sheeh\miniconda3\envs\envpt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Creating Animals Class for pytorch to access

In [3]:
class AnimalsDataset(Dataset):
    def __init__(self, data_dir, transform=None):
       self.data = ImageFolder(data_dir, transform=transform)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes    

In [13]:
dir = '../data/dataset/training_set'
dataset = AnimalsDataset(dir)
len(dataset)

target_to_class = {v: k for k, v in ImageFolder(dir).class_to_idx.items()}
print(target_to_class)

{0: 'cats', 1: 'dogs'}


In [17]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
dataset = AnimalsDataset(dir, transform=transform)
i, l = dataset[2]
i.shape

torch.Size([3, 256, 256])

 ### Creating a dataloader

In [18]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Creating Pytorch Model

In [ ]:
class AnimalClassifer(nn.Module):
    def __init__(self, num_classes=2):
        pass
    def forward(self, x):
        pass